# Introduction
Welcome to Kotlin notebook, in this environment you can run kotlin code dynamically.

## Hello World !
Let's start with the famous : "Hello World"
The last instruction of a cell code will be printed as output

In [ ]:
val greeting = "Hello Devoxx France !!"
greeting

## Notebook Outputs
You can use several output formats.

### HTML  

In [ ]:
HTML("<h1>Heading from notebook</h1>")

In [ ]:
import kotlin.random.Random
HTML("<img src='https://picsum.photos/200' />")

### Latex

In [ ]:
%use lib-ext
LATEX("F_n = F_{n-1} + F_{n-2}")

### Image Output

In [ ]:
import java.awt.Color
import java.awt.Font
import java.awt.image.BufferedImage

val image = BufferedImage(300,100,BufferedImage.TYPE_INT_ARGB)
val g = image.graphics
g.color = Color.BLACK
g.fillRect(0,0,300,100)
g.color = Color.GREEN
g.font = Font("Arial",Font.BOLD,30)
g.drawString("Hello Devoxx !!!",10, 60)
image

## Dependencies

You can use any maven dependencies in your notebook 

- With `%use` you can use predefined dependencies 
- With `@file:DependsOn("<group>:<artifact>:<version>")` you can use any maven dependency
- With USE block
```
USE {
    dependencies("<group>:<artifact>:<version>")
}
```

## DevoxxFR scraping example
The following example shows how to scrap Devoxx France 2024 schedule with [jsoup](https://jsoup.org/) and display results in a simple html table.

In [ ]:
%use datetime
@file:DependsOn("org.jsoup:jsoup:1.17.2")
import org.jsoup.Jsoup

data class Talk(val speakers:List<String>,val day:String,val slotTime: String,val sessionName:String){
    fun toTableRow():String=
        """
            <tr>
            <td>${speakers.joinToString("<br />")}</td>
            <td>$day</td>
            <td>$slotTime</td>
            <td>$sessionName</td>
            </tr>
        """.trimIndent()
}

fun scrapTalks(day:String):List<Talk>{
    val schedule = Jsoup.connect("https://www.devoxx.fr/schedule/?id=$day").get()
    return schedule.select("article.cfp-session").map {
        var speakers = it.select("div.cfp-speaker").textNodes().map { it.text() }
        val slot = it.attr("data-event-start")
        val sessionName = it.select("div.cfp-name").text()
        Talk(speakers,day,slot,sessionName)
    }
}

//Wednesday,Thursday, Friday
var talks = scrapTalks("Wednesday") 

HTML("""
<h3>Devoxx France Schedule</h3>
<table style='text-align:left'>
    <thead>
        <th>Speaker</th>
        <th>Day</th>
        <th>Slot</th>
        <th>Session</th>
    </thead>
    <tbody>
        ${talks.map { it.toTableRow() }.joinToString("\n")}
    </tbody>
</table>
""".trimIndent())
